In [1]:
from hashlib import sha256
from random import randint

In [2]:
# Parameter publik pada DSA
p = 1116674246425835791186763484099851270363
q = 558337123212917895593381742049925635181
g = 606189179136508625129240953541620883235

In [3]:
class Transaction:
    def __init__(self, inp, inpidx, inpkey, outp):
        self.inp = inp # hash dari unspent transaction yang ingin dibelanjakan
        self.inpidx = inpidx # indeks dari unspent transaction di atas yang ingin dibelanjakan
        self.inpkey = inpkey # kunci privat yang diasosiasikan dengan pemilik unspent transaction dengan indeks di atas
        self.outp = outp # penerima yang dituju & jumlah koin yang ingin ditransfer
        self.toBeHashed = str(self.inp) + str(self.inpidx) + str(self.outp) #informasi transaksi yang akan di-hash
        self.hash = sha256(self.toBeHashed.encode('utf8')).hexdigest()
        global p
        global q
        global g
        # Tanda tangan digital dengan DSA
        D = int(self.hash, 16)
        k = randint(2, q-1)
        S1 = pow(g,k,p)%q
        S2 = (D+self.inpkey*S1)*(pow(k,q-2,q))%q
        self.sign = [S1, S2]
    def __str__(self):
        string = "Unspent Transaction: " + str(self.inp) + "\n"
        string += "Index: " + str(self.inpidx) + "\n"
        string += "Transfer to: " + str(self.outp) + "\n"
        string += "Hash: " + str(self.hash) + "\n"
        string += "Signature: " + str(self.sign)
        return string

In [4]:
# Hash unspent transaction milik B sebanyak 50 koin: 1a428822908b32a273d294ad9a2a30e79b4a812d5f79cb98b73f4c406a1a3643
# Indeks dari UTXO di atas milik B: 0
# Kunci privat B: 494958898825128152883083981490204250529
# Alamat A: 498063637122669310186914979135247239947
# Alamat B: 793833148500065718802366039211618163182

inp = '1a428822908b32a273d294ad9a2a30e79b4a812d5f79cb98b73f4c406a1a3643' #hash transaksi yang mau dipakai
inpidx = 0 #index dari transaksi input yang ingin dipakai
inpkey = 494958898825128152883083981490204250529  #kunci privat yang diasosiasikan dengan address pelaku transaksi di atas
O1 = 498063637122669310186914979135247239947 #alamat A
O2 = 793833148500065718802366039211618163182 #alamat B
outp = [[O1, 35], [O2, 15]] #address yang dituju beserta jumlah yang ingin ditransfer untuk masing-masing address
transaction = Transaction(inp, inpidx, inpkey, outp)

In [5]:
print(transaction)

Unspent Transaction: 1a428822908b32a273d294ad9a2a30e79b4a812d5f79cb98b73f4c406a1a3643
Index: 0
Transfer to: [[498063637122669310186914979135247239947, 35], [793833148500065718802366039211618163182, 15]]
Hash: 3114d9838781b5d2ef9b9abaee6bc7e1531345098fcf109d7968e8710392c377
Signature: [70960175482105270864450914338080137500, 3417003949243264545771660781961748864]


In [6]:
import openpyxl

In [7]:
wb = openpyxl.load_workbook('awaiting.xlsx')
ws = wb.worksheets[0]
rows = [[str(transaction.inp),
       str(transaction.inpidx),
       str(transaction.outp),
       str(transaction.hash),
       str(transaction.sign)]]
for row in rows:
    ws.append(row)
wb.save('awaiting.xlsx')